## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


In [15]:
# Your code here - remember to use markdown cells for comments as well!

import pandas as pd
import sqlite3

In [16]:
conn = sqlite3.connect('zippedData/im.db')

df = pd.read_sql("""

SELECT *
FROM movie_basics
JOIN movie_ratings
    USING(movie_id)
JOIN movie_akas
    USING(movie_id)


""", conn)

In [17]:
df.drop_duplicates(subset=['movie_id'], inplace=True)
df.drop(columns=['primary_title', 'original_title', 'ordering', 'region', 'types', 'attributes', 'is_original_title'], inplace=True)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69577 entries, 0 to 261803
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   movie_id         69577 non-null  object 
 1   start_year       69577 non-null  int64  
 2   runtime_minutes  62830 non-null  float64
 3   genres           68937 non-null  object 
 4   averagerating    69577 non-null  float64
 5   numvotes         69577 non-null  int64  
 6   title            69577 non-null  object 
 7   language         5698 non-null   object 
dtypes: float64(2), int64(2), object(4)
memory usage: 4.8+ MB


In [25]:
df.describe()

,start_year,runtime_minutes,averagerating,numvotes
count,69577.000000,62830.000000,69577.000000,6.957700e+04
mean,2014.071791,94.645520,6.312954,3.735566e+03
std,2.530845,214.010548,1.457713,3.119919e+04
min,2010.000000,3.000000,1.000000,5.000000e+00
25%,2012.000000,81.000000,5.500000,1.500000e+01
50%,2014.000000,91.000000,6.500000,5.400000e+01
75%,2016.000000,104.000000,7.300000,3.100000e+02
max,2019.000000,51420.000000,10.000000,1.841066e+06


In [31]:
table_name_query = """SELECT name 
                      AS 'Table Names' 
                      FROM sqlite_master 
                      WHERE type='table';"""

pd.read_sql(table_name_query, conn)

,Table Names
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


In [37]:
df3 = pd.read_sql("""

SELECT *
FROM directors
JOIN persons
USING(person_id);

""", conn)

In [41]:
df3.drop_duplicates(subset=['person_id'], inplace=True)

In [44]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109251 entries, 0 to 291169
Data columns (total 6 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   movie_id            109251 non-null  object 
 1   person_id           109251 non-null  object 
 2   primary_name        109251 non-null  object 
 3   birth_year          16961 non-null   float64
 4   death_year          628 non-null     float64
 5   primary_profession  108634 non-null  object 
dtypes: float64(2), object(4)
memory usage: 5.8+ MB


In [46]:
df3.drop(columns=['death_year', 'birth_year'], inplace=True)

In [47]:
df3

,movie_id,person_id,primary_name,primary_profession
0,tt0285252,nm0899854,Tony Vitale,"producer,director,writer"
1,tt0462036,nm1940585,Bill Haley,"director,writer,producer"
2,tt0835418,nm0151540,Jay Chandrasekhar,"director,actor,writer"
4,tt0878654,nm0089502,Albert Pyun,"director,writer,producer"
5,tt0878654,nm2291498,Joe Baile,"producer,director,camera_department"
...,...,...,...,...
291164,tt8999892,nm10122247,C. Damon Adcock,None
291166,tt8999974,nm10122357,Daysi Burbano,"director,writer,cinematographer"
291167,tt9001390,nm6711477,Bernard Lessa,"director,writer,cinematographer"
291168,tt9001494,nm10123242,Tate Nova,"director,producer"


In [48]:
df4 = pd.read_csv('zippedData/bom.movie_gross.csv.gz', compression='gzip')

In [49]:
df4

,title,studio,domestic_gross,foreign_gross,year
0,Toy Story 3,BV,415000000.0,652000000,2010
1,Alice in Wonderland (2010),BV,334200000.0,691300000,2010
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000,2010
3,Inception,WB,292600000.0,535700000,2010
4,Shrek Forever After,P/DW,238700000.0,513900000,2010
...,...,...,...,...,...
3382,The Quake,Magn.,6200.0,NaN,2018
3383,Edward II (2018 re-release),FM,4800.0,NaN,2018
3384,El Pacto,Sony,2500.0,NaN,2018
3385,The Swan,Synergetic,2400.0,NaN,2018


In [50]:
df5 = pd.read_csv('zippedData/tn.movie_budgets.csv.gz', compression='gzip')

In [51]:
df5

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"
...,...,...,...,...,...,...
5777,78,"Dec 31, 2018",Red 11,"$7,000",$0,$0
5778,79,"Apr 2, 1999",Following,"$6,000","$48,482","$240,495"
5779,80,"Jul 13, 2005",Return to the Land of Wonders,"$5,000","$1,338","$1,338"
5780,81,"Sep 29, 2015",A Plague So Pleasant,"$1,400",$0,$0


In [52]:
df6 = pd.read_csv('zippedData/tmdb.movies.csv.gz', compression='gzip')

In [61]:
df6['movie'] = df6.original_title

In [62]:
df6

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count,movies,movie
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788,Harry Potter and the Deathly Hallows: Part 1,Harry Potter and the Deathly Hallows: Part 1
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610,How to Train Your Dragon,How to Train Your Dragon
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368,Iron Man 2,Iron Man 2
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174,Toy Story,Toy Story
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186,Inception,Inception
...,...,...,...,...,...,...,...,...,...,...,...,...
26512,26512,"[27, 18]",488143,en,Laboratory Conditions,0.600,2018-10-13,Laboratory Conditions,0.0,1,Laboratory Conditions,Laboratory Conditions
26513,26513,"[18, 53]",485975,en,_EXHIBIT_84xxx_,0.600,2018-05-01,_EXHIBIT_84xxx_,0.0,1,_EXHIBIT_84xxx_,_EXHIBIT_84xxx_
26514,26514,"[14, 28, 12]",381231,en,The Last One,0.600,2018-10-01,The Last One,0.0,1,The Last One,The Last One
26515,26515,"[10751, 12, 28]",366854,en,Trailer Made,0.600,2018-06-22,Trailer Made,0.0,1,Trailer Made,Trailer Made


In [64]:
merge_df = df6.merge(df5, on='movie')